In [1]:
# 000_UTILIDADES.ipynb
# %% [markdown]
"""
# 000.UTILIDADES: Copiar Pastas no Google Drive por ID

Este notebook copia diretamente, no servidor do Google Drive, todo o conteúdo de uma pasta de origem
enquanto preserva subdiretórios, para uma pasta de destino, usando remotes temporários do Rclone e
o flag --drive-server-side-across-configs.
"""
# %% [code]
import os
import subprocess
import tempfile
from configparser import ConfigParser

# %% [code]
# 1) Solicitar IDs ao usuário
drive = os.path.expanduser("~/.config/rclone/rclone.conf")
print("--- Copier Pastas Google Drive ---")
origin_id = input("ID_DA_PASTA_ORIGEM (ex: 1THttiv0Q0ONduDNQhCnvlS8geqVD3FRs): ").strip()
dest_id   = input("ID_DA_PASTA_DESTINO (ex: 1Mj1MifracuE5jQEeiMJsjr0TNLct5xx-): ").strip()

# %% [code]
# 2) Carregar config existente e preparar remotes temporários
cfg = ConfigParser()
cfg.read(drive)
base = cfg.get('gdrive', 'type') and cfg['gdrive']  # base config
# src remote
cfg['src'] = dict(cfg['gdrive'])
cfg['src']['root_folder_id'] = origin_id
# dst remote
cfg['dst'] = dict(cfg['gdrive'])
cfg['dst']['root_folder_id'] = dest_id

# 3) Gravar config temporário
tmp_conf = tempfile.NamedTemporaryFile(delete=False, suffix=".conf").name
with open(tmp_conf, 'w') as f:
    cfg.write(f)
print(f"Arquivo de config temporário gerado: {tmp_conf}")

# %% [code]
# 4) Executar cópia server-side entre src: e dst:
cmd = [
    'rclone', '--config', tmp_conf,
    'copy', 'src:', 'dst:',
    '--drive-server-side-across-configs',
    '--create-empty-src-dirs'
]
print("Executando:", ' '.join(cmd))
res = subprocess.run(cmd, capture_output=True, text=True)
if res.returncode == 0:
    print("✓ Copia concluída com sucesso!")
else:
    print(f"✗ Erro na cópia (code {res.returncode}):")
    print(res.stderr)

# %% [code]
# 5) Listar conteúdo da pasta de destino para verificação
print("\nConteúdo da pasta de destino:")
subprocess.run(['rclone', '--config', tmp_conf, 'ls', 'dst:'])




--- Copier Pastas Google Drive ---
Arquivo de config temporário gerado: /tmp/tmp5rqc2k77.conf
Executando: rclone --config /tmp/tmp5rqc2k77.conf copy src: dst: --drive-server-side-across-configs --create-empty-src-dirs
✓ Copia concluída com sucesso!

Conteúdo da pasta de destino:
   162375 Realizando análise do repositório.pdf
    28549 Status em 160425_15h53min.docx
   211879 Política de Curadoria de Dados.pdf
   350180 Política de Curadoria de Dados.docx
    57936 unnamed.png
    29721 CONTEXTOS E DOCUMENTOS DE INFORMAÇÃO PARA LLM/INVENTÁRIO DO QUE FOI FEITO EM 12_04.docx
   139654 CONTEXTOS E DOCUMENTOS DE INFORMAÇÃO PARA LLM/Resumo do Ambiente Operacional e Objetivos para o Próximo Chat.pdf
    28743 CONTEXTOS E DOCUMENTOS DE INFORMAÇÃO PARA LLM/Resumo do Ambiente Operacional e Objetivos para o Próximo Chat.docx


CompletedProcess(args=['rclone', '--config', '/tmp/tmp5rqc2k77.conf', 'ls', 'dst:'], returncode=0)